In [4]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import xgboost as xgb
from numpy import linalg
import socket
from scipy.stats import zscore, mode
import json
import time

In [ ]:
def pm(angle):
    if angle > 2*np.pi:
        while angle > np.pi:
            angle -= 2*np.pi
    if angle < -2*np.pi:
        while angle < -np.pi:
            angle += 2*np.pi
    return angle

pm = np.vectorize(pm)


wave_dist= (299_792_458/(2.4e6+270e3))/50
#phase_diff - 29= np.sin(np.deg2rad(angles))*2*np.pi/wave_dist
aoa = lambda pds: np.rad2deg( np.arcsin(np.clip(pm(pds[1:] - pds[:-1] -np.deg2rad(2*99)) *wave_dist/(2*np.pi),-1,1))) 

# Classifier

In [27]:
model = xgb.XGBRegressor()
model.load_model("./regressor_pd_row.json")

azi_ma = [0]*6
el_ma = [0]*6

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
    
def on_message(client, userdata, msg):
    
    data = json.loads(msg.payload)
    
    phases = np.array([np.angle(complex(data[1]["aoa"]["iq"][i],data[1]["aoa"]["iq"][i+1])) for i in range(0,len(data[1]["aoa"]["iq"]),2)])
        
    pds = np.array([phases[i:i+8][1:]-phases[i:i+8][:-1] for i in range(8,len(phases),8)][:-1])

    model_predictions = model.predict(pds)
    
    azi = model_predictions[0::2]
    el = model_predictions[1::2]


    azi_ma.append(np.mean(azi))
    azi_ma.pop(0) 

    el_ma.append(float(np.mean(el)))
    el_ma.pop(0) 

    aoa_xz = np.arctan2(np.deg2rad(np.mean(azi_ma)),np.deg2rad(np.mean(el_ma))) 

    print(f"Elevation I: {el_ma[-1]} \t Azimutal I: {azi_ma[-1]}")
    print(f"Elevation: {np.mean(el_ma)} \t Azimutal: {np.mean(azi_ma)}")
    print(f"AoA={np.rad2deg(aoa_xz) +180} \n")

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)
#client.connect("test.mosquitto.org", 1883, 60)

client.loop_forever()

Conectado com o código de resultado 0
Elevation I: -17.84133529663086 	 Azimutal I: -37.331398010253906
Elevation: -2.97355588277181 	 Azimutal: -6.221899668375651
AoA=64.45603966748888 

Elevation I: -11.015708923339844 	 Azimutal I: -36.1920051574707
Elevation: -4.809507369995117 	 Azimutal: -12.253900527954102
AoA=68.57059470945015 

Elevation I: -12.496817588806152 	 Azimutal I: -34.50564956665039
Elevation: -6.8923103014628095 	 Azimutal: -18.004842122395832
AoA=69.05299659731351 

Elevation I: -1.384286880493164 	 Azimutal I: 15.664461135864258
Elevation: -7.123024781545003 	 Azimutal: -15.394098599751791
AoA=65.16948190050724 

Elevation I: -5.779555797576904 	 Azimutal I: 15.539959907531738
Elevation: -8.086284081141153 	 Azimutal: -12.804105281829834
AoA=57.72601671608855 

Elevation I: -16.434661865234375 	 Azimutal I: -28.757823944091797
Elevation: -10.82539439201355 	 Azimutal: -17.597076416015625
AoA=58.4009419876934 

Elevation I: -19.656841278076172 	 Azimutal I: -44.382

KeyboardInterrupt: 

In [21]:
[[i,i+8] for i in range(8,74,8)]

[[8, 16],
 [16, 24],
 [24, 32],
 [32, 40],
 [40, 48],
 [48, 56],
 [56, 64],
 [64, 72],
 [72, 80]]